# Coastal Area simulation using SWASH with the Inductiva API

The Inductiva API provides an array of open source numerical simulators ready to use in the cloud, all from within Python code.
A popular simulator related to Coastal Dynamics is SWASH, which we'll use to demonstrate a coastal area simulation using the Inductiva API.
This simulation scenario consists of waves propagating in a coastal area represented by a bathymetric profile (i.e., the map of depths of the sea bottom as a function of spatial coordinates x, y). Waves are injected into the domain from one of the boundaries of the simulation with a given amplitude and period.

The notebook is organized as follows: first, we'll take a look at the input files required to run the simulation. We'll then run the simulation in Cloud machines, and then take a look at the output files.


### Tips for running the notebook

For this notebook, you'll need to have a few Python packages installed, which you can do 

```pip install inductiva scipy numpy matplotlib```

If you're on Google Colab, run the the following cells to install the required dependencies and download the necessary files.

In [ ]:
!pip install inductiva scipy numpy matplotlib
!curl https://storage.googleapis.com/inductiva-api-demo-files/code-blue-hackathon.zip
!unzip code-blue-hackathon.zip

## 1. Prepare the simulation inputs

For this simulation, we'll require two input files:
- The [bathymetry](assets/coastal_area_simulation_demo/bathymetry.bot), which is a simple text file with a matrix of depths (in meters).
- The [simulator's configuration file](assets/coastal_area_simulation_demo/input.sws), which specifies other parameters (such as wave amplitude, period, the computational grid, among others). Lines starting with `$` are comments giving notes on some of the input values.

The following code block loads the provided bathymetry file into a numpy array, and inspects some information about it.


In [ ]:
import numpy as np

# The bathymetry data for a SWASH simulation is stored in a simple text file
# with a matrix of depths in meters. The first and second dimensions of the
# matrix correspond to the x and y coordinates of the grid points, respectively.
# Let's load the data into a numpy array and see some information about it.
bathymetry = np.loadtxt('assets/coastal_area_simulation_demo/bathymetry.bot')

print('Shape of the grid:', bathymetry.shape)
print('Max depth (m):', np.max(bathymetry))
print('Max height (m):', - np.min(bathymetry))

You can see the bathymetry grid has 199x169 points. This specific bathymetry was sampled with a resolution of 5m x 5m (*i.e.*, each grid point is separated by 5m in each direction to adjacent points).
Let's plot the bathymetry using the `matplotlib` library.

In [ ]:
import matplotlib.pyplot as plt

def visualize_bathymetry(bathymetry, x_res, y_res):
    # Specify the size of the image to match the size of the grid in meters
    extent = (0, x_res*bathymetry.shape[0], 0, y_res*bathymetry.shape[1])

    fig = plt.figure()
    ax = fig.add_subplot()
    im = ax.imshow(
        -1 * bathymetry.transpose(),
        cmap='coolwarm',
        origin='lower',
        extent=extent,
    )
    ax.set(aspect='equal', xlabel='$x$ [m]', ylabel='$y$ [m]')

    fig.colorbar(im, ax=ax, label="Height [m]")

# The provided bathymetry is sampled at 5m x 5m resolution
visualize_bathymetry(bathymetry, x_res=5, y_res=5)

## 2. Run the simulation

Now that we have input files for the simulation, let's run the actual simulation.
We'll create a virtual machine in the Cloud, run the simulation in that machine, and download the outputs of the simulation, all with a few simple lines of Python code.

You should have already registered an account in the Inductiva API. If not, you can do it [here](https://console.inductiva.ai/). Afterwards, go to the `welcome` tab where you'll find an API Key. 
You should copy it and paste in the following cell, replacing `<YOUR_API_KEY_HERE>`.

We'll then import the `inductiva` package. Note that you must run the API Key cell before importing the `inductiva` package.

In [ ]:
%env INDUCTIVA_API_KEY=<YOUR_API_KEY_HERE>

In [ ]:
import inductiva

To run the simulation, we'll first need to create a machine in the Cloud that's dedicated for you. 
Sounds complicated, but with the Inductiva API, you don't need Cloud expertise to leverage the Cloud's computational power for your simulation workloads.
Machines in the Inductiva API are managed via the "Machine Group" concept.
By instantiating an object of the class `MachineGroup`, you're requesting a machine group with a given configuration. In this case, we'll
request a machine group with 1 machine of type `e2-standard-4`. This is a machine type provided by Google Cloud which has 4 vCPUs. While this machine is fairly weak, it is good enough for this simple demo simulation. To run larger simulations, you could launch a machine group with hundreds of cores just as easily.
By calling the method `start()` of the `machine_group` object, we are requesting the machines to actually start running. 
After the machines start, they'll be ready to run simulations.

In [ ]:
machine_group = inductiva.resources.MachineGroup(machine_type="e2-standard-4", num_machines=1)
machine_group.start()

Now that we have a machine ready, we can submit our SWASH simulation.
For that, let's create a `swash` object and call it's run method. There are three arguments to specify:
- `input_dir`: the path to the directory in your local computer which has the simulator input files.
- `sim_config_filename`: the filename of the simulator configuration file. This file must exist inside the `input_dir`.
- `on`: this machine group where the simulation will run. 

In [ ]:
swash = inductiva.simulators.SWASH()

task = swash.run(
    input_dir="assets/coastal_area_simulation_demo",
    sim_config_filename="input.sws",
    on=machine_group,
)

The `run` method returns an object which represents the newly submitted simulation task. We can call the `wait` method to wait until the simulation is complete. It should take only a few seconds.

In [ ]:
task.wait()

Now that the simulation is over, we download the output files generated during the simulation.
Besides the output files generated by the simulator, you'll also find the logs of the simulator command (in the `stdout.txt` and `stderr.txt` files).

In [ ]:
output_path = task.download_outputs()

Let's take a look at the downloaded files:

In [ ]:
!ls {output_path}

Now that we've successfuly run the simulation and no longer need the machine group, we can terminate it, by running the following method.

In [ ]:
machine_group.terminate()

## 3. Analyse the outputs

Now that we have successfully downloaded the output files of the simulation, we can analyse some of the results.
The resulting physical properties are stored in the files with `.mat` extension (binary MATLAB files).
Let's load the `water_level.mat` file with `scipy`.

In [ ]:
import scipy
import scipy.io

water_level_dict = scipy.io.loadmat(f"{output_path}/water_level.mat")
print(water_level_dict.keys())

The `water_level.mat` file includes the water level (in meters) for each simulation timestep.
The simulation timestep is specified in the suffix of the format `hhmmss_ms` in the keys of the dictionary printed above.
Let's load all the water level grids and corresponding timesteps into lists.

In [12]:
timesteps = []
water_level_data = []

for key, data in water_level_dict.items():

    if not key.startswith("Watlev"):
        continue

    time_ms = "".join(key.split("_")[1:])

    # Convert time to seconds
    time_s = float(time_ms[:2]) * 3600. + \
        float(time_ms[2:4]) * 60. + \
        float(time_ms[4:6]) +  \
        float(time_ms[6:]) / 1000.

    timesteps.append(time_s)
    water_level_data.append(data.transpose())


We can now, for instance, plot the water level over time in a given point of the bathymetry grid.

In [ ]:
def plot_water_level_over_time(water_level_data, timesteps, x, y):
    water_level_xy = [mat[x, y] for mat in water_level_data]

    plt.figure(figsize=(10, 6))
    plt.plot(timesteps, water_level_xy, label=f'Water Level at {(x, y)}')
    plt.xlabel('Time (s)')
    plt.ylabel('Water Level (m)')
    plt.title(f'Water Level Over Time at {(x, y)}')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_water_level_over_time(water_level_data, timesteps, 100, 10)